In [ ]:
import pandas as pd

## Compare two submissions. Replace without_fallbacks with a file you want to compare
## to DEADGE0825_fallbacks.csv (our best result thus far).

prediction_mapping = pd.read_csv('../data/prediction_mapping.csv')
your_submission = pd.read_csv('DEADGE_submission0825.csv')
best_submission_with_fallbacks = pd.read_csv('DEADGE_submission0825_tweedie_fallbacks.csv')

your_submission = your_submission[your_submission["predicted_weight"]>0]
best_submission_with_fallbacks = best_submission_with_fallbacks[best_submission_with_fallbacks["predicted_weight"]>0]

your_submission = your_submission.merge(prediction_mapping, on="ID", how="left")

test2024_agg = your_submission.groupby("rm_id", as_index=False).agg({
        "predicted_weight": "max",
        }).sort_values("predicted_weight", ascending=False)
deadge2025_agg = best_submission_with_fallbacks.groupby("rm_id", as_index=False).agg({
        "predicted_weight": "max",
        }).sort_values("predicted_weight", ascending=False) 


merged = test2024_agg.merge(deadge2025_agg, on="rm_id", suffixes=("_your_submission", "_best_submission"), how="outer")

# sort merged by predicted_weight_your_submission descending
merged = merged.sort_values("predicted_weight_your_submission", ascending=False)
print(merged)

    rm_id  predicted_weight_your_submission  predicted_weight_best_submission
2    2130                      7.810472e+06                      7.810472e+06
24   3865                      4.966045e+06                      4.966045e+06
15   3126                      3.653246e+06                      3.653246e+06
23   3781                      3.054471e+06                      3.054471e+06
14   3125                      2.255302e+06                      2.255302e+06
11   3122                      1.985849e+06                      1.985849e+06
13   3124                      1.663868e+06                      1.663868e+06
12   3123                      1.551946e+06                      1.551946e+06
17   3282                      9.542247e+05                      9.542247e+05
25   3901                      8.880812e+05                      8.880812e+05
7    2142                      1.794663e+05                      1.794663e+05
20   3421                      7.397329e+04                     

In [2]:
### If you have a submission which needs fallbacks,
### use the code below to determine which rm_ids need fallbacks

best_without_fallbacks_name = 'DEADGE_submission0825'
fallbacks_name = 'Tweedie_processed'
best_with_fallbacks_file_name = f"{best_without_fallbacks_name}_tweedie_fallbacks"

best_without_fallbacks_df = pd.read_csv(f"{best_without_fallbacks_name}.csv")
fallbacks_df = pd.read_csv(f"{fallbacks_name}.csv")

# Find latest ID for each rm_id in prediction_mapping
latest_ids = prediction_mapping.groupby("rm_id")["ID"].max().reset_index()
highest_predicted_weights = best_without_fallbacks_df[best_without_fallbacks_df["ID"].isin(latest_ids["ID"])].sort_values("predicted_weight", ascending=False).reset_index(drop=True)

# print(highest_predicted_weights.head(15))
# print(latest_ids.head())

# good rmids are rmids where the latest id prediction is greater than 100
good_rmids = latest_ids[latest_ids["ID"].isin(highest_predicted_weights[highest_predicted_weights["predicted_weight"] > 100]["ID"])]["rm_id"].tolist()

fallbacks_nonzero_rmids = prediction_mapping[prediction_mapping["ID"].isin(fallbacks_df[fallbacks_df["predicted_weight"]>0]["ID"])]["rm_id"].unique().tolist()
fallback_rmids = set(fallbacks_nonzero_rmids) - set(good_rmids)
print("Fallback rm_ids:", sorted(fallback_rmids))


Fallback rm_ids: [2125, 2129, 2131, 2132, 2133, 2134, 2135, 2143, 2144, 2145, 2161, 2741, 3201, 3362, 3381, 3601, 3621, 3883, 4081, 4161, 4222, 4263, 4302]


In [3]:
best_without_fallbacks_df = best_without_fallbacks_df.merge(prediction_mapping, on="ID", how="left")

### David has somehow found a good list of fallbacks
### These perform better than the ones you get by simply taking all the ones which appear in 2024, and which are not in good_rmids
david_fallbacks = [2124, 2131, 2133, 2134, 2135, 2145, 2161, 3201, 3362, 3381, 3581, 3601, 3621, 4044, 4081, 4161, 4222, 4302]
christian_additions = [2143, 387, 1872] # These are rm_ids where QuantileLoss tft predicts in range (0, 10]
david_fallbacks.extend(christian_additions)
david_fallbacks_relevant = set(david_fallbacks) - set(good_rmids)

david = True
fallbacks = david_fallbacks_relevant if david else fallback_rmids

for rm_id in fallbacks:
    # Find IDs corresponding to this rm_id
    ids = prediction_mapping[prediction_mapping["rm_id"] == rm_id]["ID"].tolist()
    for id in ids:
        # Put predicted_weight from test2024 into best
        weight = fallbacks_df[fallbacks_df["ID"] == id]["predicted_weight"]
        best_without_fallbacks_df.loc[best_without_fallbacks_df["ID"] == id, "predicted_weight"] = weight

print(f"Added fallbacks for rm_ids: {sorted(fallbacks)}\nSaving to {best_with_fallbacks_file_name}")
best_without_fallbacks_df.to_csv(f"{best_with_fallbacks_file_name}.csv", index=False)

Added fallbacks for rm_ids: [387, 1872, 2124, 2131, 2133, 2134, 2135, 2143, 2145, 2161, 3201, 3362, 3381, 3581, 3601, 3621, 4044, 4081, 4161, 4222, 4302]
Saving to DEADGE_submission0825_tweedie_fallbacks
